In [ ]:
import requests
import io
import base64
import utils
import random
from PIL import Image
from pathlib import Path

API_URL = "http://127.0.0.1:7861/sdapi/v1/txt2img"
SPLIT = 0.8
ITERATIONS = 10000
START = 0

PROMPTS_PATH = "imagenet1k.json"

DATASET = "imagenet1k"
TRAIN_DIRECTORY = Path(f"outputs/{DATASET}/train/fake")
TRAIN_DIRECTORY.mkdir(parents=True, exist_ok=True)

TEST_DIRECTORY = Path(f"outputs/{DATASET}/test/fake")
TEST_DIRECTORY.mkdir(parents=True, exist_ok=True)

ITERATION_THRESHOLD = int(SPLIT * ITERATIONS)

In [ ]:
prompts = utils.get_prompts(PROMPTS_PATH)
print(prompts[:10])
print(len(prompts))

In [ ]:
for i in range(START, ITERATIONS):
  print(f"Generating image: {i + 1}/{ITERATIONS}")
  prompt = random.choice(prompts)
  payload = utils.prepare_payload(prompt, amount=1)

  response = requests.post(API_URL, json=payload).json()

  for index, image in enumerate(response["images"]):
    image = Image.open(io.BytesIO(base64.b64decode(image)))
    
    if i < ITERATION_THRESHOLD:
      save_path = TRAIN_DIRECTORY / f"{i}-{index}-{prompt}.png"
    else:
      save_path = TEST_DIRECTORY / f"{i}-{index}-{prompt}.png"
      
    image.save(save_path)